# Importação de bibliotecas e funções

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import warnings
import math
import matplotlib.ticker as ticker
warnings.filterwarnings('ignore') #ignora os avisos

In [ ]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [ ]:
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
def media_movel(serie_temporal, periodos = 7):
  serie = serie_temporal.copy()
  n = len(serie)
  pe = periodos

  MM = []

  for i in range(0,pe):
    valor = sum(serie[0:i+1])/(i+1)
    MM.append(valor)

  for i in range(pe,n):
    valor = sum(serie[i-pe:i])/pe
    MM.append(valor)
  
  return MM

In [ ]:
def media_movel_exp(serie_temporal, periodos = 7):
  serie = serie_temporal.copy()
  n = len(serie)
  pe = periodos
  mu = 2/(pe+1)

  MME = [serie[0]]

  for i in range(1,n):
    valor = mu*(serie[i] - MME[i-1]) + MME[i-1]
    MME.append(valor)

  return MME

# Importação e tratamento inicial dos dados

In [ ]:
colunas = ['Municipio',
           'Cod_IBGE',
           'Data',
           'Sem_epidemiologica',
           'Pop_estimada',
           'Pop_estimada_2019',
           'Ultimo?',
           'Repetido?',
           'Ultimos_confirmados',
           'Ultimos_confirmados_por_100k_habitantes',
           'Ultima_data',
           'Ultima_taxa_morte',
           'Ultimas_mortes',
           'Ordem_por_lugar',
           'Tipo_lugar',
           'Estado',
           'Novos_casos',
           'Novas_mortes'
           
           ]

In [ ]:
url = 'https://raw.githubusercontent.com/renanmath/Bootcamp_Projeto_Modulo03/main/dados/covid_CE'

In [ ]:
dados_geral = pd.read_csv(url)
dados_geral.drop('Unnamed: 0', axis = 1, inplace=True)
dados_geral.columns = colunas

In [ ]:
dados_geral

,Municipio,Cod_IBGE,Data,Sem_epidemiologica,Pop_estimada,Pop_estimada_2019,Ultimo?,Repetido?,Ultimos_confirmados,Ultimos_confirmados_por_100k_habitantes,Ultima_data,Ultima_taxa_morte,Ultimas_mortes,Ordem_por_lugar,Tipo_lugar,Estado,Novos_casos,Novas_mortes
0,Aquiraz,"2,301,000.00",2020-03-16,202012,"80,935.00","80,271.00",False,False,1,1.24,2020-03-16,0.00,0,1,city,CE,1,0
1,Fortaleza,"2,304,400.00",2020-03-16,202012,"2,686,612.00","2,669,342.00",False,False,8,0.30,2020-03-16,0.00,0,1,city,CE,8,0
2,NaN,23.00,2020-03-16,202012,"9,187,103.00","9,132,078.00",False,False,9,0.10,2020-03-16,0.00,0,1,state,CE,9,0
3,Aquiraz,"2,301,000.00",2020-03-17,202012,"80,935.00","80,271.00",False,False,1,1.24,2020-03-17,0.00,0,2,city,CE,0,0
4,Fortaleza,"2,304,400.00",2020-03-17,202012,"2,686,612.00","2,669,342.00",False,False,9,0.33,2020-03-17,0.00,0,2,city,CE,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79649,Uruoca,"2,313,906.00",2021-06-20,202125,"13,915.00","13,840.00",False,True,1426,"10,247.93",2021-06-19,0.02,31,428,city,CE,0,0
79650,Varjota,"2,313,955.00",2021-06-20,202125,"18,471.00","18,420.00",False,True,2279,"12,338.26",2021-06-19,0.01,27,413,city,CE,0,0
79651,Viçosa do Ceará,"2,314,102.00",2021-06-20,202125,"61,410.00","60,889.00",False,True,4788,"7,796.78",2021-06-19,0.03,140,428,city,CE,0,0
79652,Várzea Alegre,"2,314,003.00",2021-06-20,202125,"40,903.00","40,721.00",False,True,3076,"7,520.23",2021-06-19,0.02,66,438,city,CE,0,0


In [ ]:
dados_geral.dtypes


Municipio                                   object
Cod_IBGE                                   float64
Data                                        object
Sem_epidemiologica                           int64
Pop_estimada                               float64
Pop_estimada_2019                          float64
Ultimo?                                       bool
Repetido?                                     bool
Ultimos_confirmados                          int64
Ultimos_confirmados_por_100k_habitantes    float64
Ultima_data                                 object
Ultima_taxa_morte                          float64
Ultimas_mortes                               int64
Ordem_por_lugar                              int64
Tipo_lugar                                  object
Estado                                      object
Novos_casos                                  int64
Novas_mortes                                 int64
dtype: object

In [ ]:
dados_geral['Data'] = pd.to_datetime(dados_geral['Data'], format= '%Y-%m-%d')

In [ ]:
dados_geral

,Municipio,Cod_IBGE,Data,Sem_epidemiologica,Pop_estimada,Pop_estimada_2019,Ultimo?,Repetido?,Ultimos_confirmados,Ultimos_confirmados_por_100k_habitantes,Ultima_data,Ultima_taxa_morte,Ultimas_mortes,Ordem_por_lugar,Tipo_lugar,Estado,Novos_casos,Novas_mortes
0,Aquiraz,"2,301,000.00",2020-03-16,202012,"80,935.00","80,271.00",False,False,1,1.24,2020-03-16,0.00,0,1,city,CE,1,0
1,Fortaleza,"2,304,400.00",2020-03-16,202012,"2,686,612.00","2,669,342.00",False,False,8,0.30,2020-03-16,0.00,0,1,city,CE,8,0
2,NaN,23.00,2020-03-16,202012,"9,187,103.00","9,132,078.00",False,False,9,0.10,2020-03-16,0.00,0,1,state,CE,9,0
3,Aquiraz,"2,301,000.00",2020-03-17,202012,"80,935.00","80,271.00",False,False,1,1.24,2020-03-17,0.00,0,2,city,CE,0,0
4,Fortaleza,"2,304,400.00",2020-03-17,202012,"2,686,612.00","2,669,342.00",False,False,9,0.33,2020-03-17,0.00,0,2,city,CE,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79649,Uruoca,"2,313,906.00",2021-06-20,202125,"13,915.00","13,840.00",False,True,1426,"10,247.93",2021-06-19,0.02,31,428,city,CE,0,0
79650,Varjota,"2,313,955.00",2021-06-20,202125,"18,471.00","18,420.00",False,True,2279,"12,338.26",2021-06-19,0.01,27,413,city,CE,0,0
79651,Viçosa do Ceará,"2,314,102.00",2021-06-20,202125,"61,410.00","60,889.00",False,True,4788,"7,796.78",2021-06-19,0.03,140,428,city,CE,0,0
79652,Várzea Alegre,"2,314,003.00",2021-06-20,202125,"40,903.00","40,721.00",False,True,3076,"7,520.23",2021-06-19,0.02,66,438,city,CE,0,0


In [ ]:
dados_geral.dtypes

Municipio                                          object
Cod_IBGE                                          float64
Data                                       datetime64[ns]
Sem_epidemiologica                                  int64
Pop_estimada                                      float64
Pop_estimada_2019                                 float64
Ultimo?                                              bool
Repetido?                                            bool
Ultimos_confirmados                                 int64
Ultimos_confirmados_por_100k_habitantes           float64
Ultima_data                                        object
Ultima_taxa_morte                                 float64
Ultimas_mortes                                      int64
Ordem_por_lugar                                     int64
Tipo_lugar                                         object
Estado                                             object
Novos_casos                                         int64
Novas_mortes  

In [ ]:
dados_CE = dados_geral[dados_geral['Tipo_lugar'] == 'state']
dados_CE

,Municipio,Cod_IBGE,Data,Sem_epidemiologica,Pop_estimada,Pop_estimada_2019,Ultimo?,Repetido?,Ultimos_confirmados,Ultimos_confirmados_por_100k_habitantes,Ultima_data,Ultima_taxa_morte,Ultimas_mortes,Ordem_por_lugar,Tipo_lugar,Estado,Novos_casos,Novas_mortes
2,NaN,23.00,2020-03-16,202012,"9,187,103.00","9,132,078.00",False,False,9,0.10,2020-03-16,0.00,0,1,state,CE,9,0
5,NaN,23.00,2020-03-17,202012,"9,187,103.00","9,132,078.00",False,False,10,0.11,2020-03-17,0.00,0,2,state,CE,1,0
9,NaN,23.00,2020-03-18,202012,"9,187,103.00","9,132,078.00",False,False,19,0.21,2020-03-18,0.00,0,3,state,CE,9,0
14,NaN,23.00,2020-03-19,202012,"9,187,103.00","9,132,078.00",False,False,24,0.26,2020-03-19,0.00,0,4,state,CE,5,0
21,NaN,23.00,2020-03-20,202012,"9,187,103.00","9,132,078.00",False,False,68,0.74,2020-03-20,0.00,0,5,state,CE,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78909,NaN,23.00,2021-06-16,202124,"9,187,103.00","9,132,078.00",False,False,856267,"9,320.32",2021-06-16,0.03,21876,458,state,CE,3679,177
79095,NaN,23.00,2021-06-17,202124,"9,187,103.00","9,132,078.00",False,False,858502,"9,344.64",2021-06-17,0.03,21901,459,state,CE,2235,25
79281,NaN,23.00,2021-06-18,202124,"9,187,103.00","9,132,078.00",False,False,861688,"9,379.32",2021-06-18,0.03,21967,460,state,CE,3186,66
79467,NaN,23.00,2021-06-19,202124,"9,187,103.00","9,132,078.00",False,False,864529,"9,410.25",2021-06-19,0.03,22043,461,state,CE,2841,76


Criando as médias móveis

In [ ]:
dados_CE['MM7_casos_acumulados'] = media_movel(dados_CE['Ultimos_confirmados'].tolist(), periodos=7)
dados_CE['MM28_casos_acumulados'] = media_movel(dados_CE['Ultimos_confirmados'].tolist(), periodos=28)
dados_CE['MM7_obitos_acumulados'] = media_movel(dados_CE['Ultimas_mortes'].tolist(), periodos=7)
dados_CE['MM28_obitos_acumulados'] = media_movel(dados_CE['Ultimas_mortes'].tolist(), periodos=28)
dados_CE['MM7_casos_novos'] = media_movel(dados_CE['Novos_casos'].tolist(), periodos=7)
dados_CE['MM28_casos_novos'] = media_movel(dados_CE['Novos_casos'].tolist(), periodos=28)
dados_CE['MM7_obitos_novos'] = media_movel(dados_CE['Novas_mortes'].tolist(), periodos=7)
dados_CE['MM28_obitos_novos'] = media_movel(dados_CE['Novas_mortes'].tolist(), periodos=28)

#dados_CE['MM7_casos_novos'] = dados_CE['Novos_casos'].rolling(window = 7, center = False).mean()

Semana epidemiológica

In [ ]:
dados_CE

,Municipio,Cod_IBGE,Data,Sem_epidemiologica,Pop_estimada,Pop_estimada_2019,Ultimo?,Repetido?,Ultimos_confirmados,Ultimos_confirmados_por_100k_habitantes,Ultima_data,Ultima_taxa_morte,Ultimas_mortes,Ordem_por_lugar,Tipo_lugar,Estado,Novos_casos,Novas_mortes,MM7_casos_acumulados,MM28_casos_acumulados,MM7_obitos_acumulados,MM28_obitos_acumulados,MM7_casos_novos,MM28_casos_novos,MM7_obitos_novos,MM28_obitos_novos
2,NaN,23.00,2020-03-16,202012,"9,187,103.00","9,132,078.00",False,False,9,0.10,2020-03-16,0.00,0,1,state,CE,9,0,9.00,9.00,0.00,0.00,9.00,9.00,0.00,0.00
5,NaN,23.00,2020-03-17,202012,"9,187,103.00","9,132,078.00",False,False,10,0.11,2020-03-17,0.00,0,2,state,CE,1,0,9.50,9.50,0.00,0.00,5.00,5.00,0.00,0.00
9,NaN,23.00,2020-03-18,202012,"9,187,103.00","9,132,078.00",False,False,19,0.21,2020-03-18,0.00,0,3,state,CE,9,0,12.67,12.67,0.00,0.00,6.33,6.33,0.00,0.00
14,NaN,23.00,2020-03-19,202012,"9,187,103.00","9,132,078.00",False,False,24,0.26,2020-03-19,0.00,0,4,state,CE,5,0,15.50,15.50,0.00,0.00,6.00,6.00,0.00,0.00
21,NaN,23.00,2020-03-20,202012,"9,187,103.00","9,132,078.00",False,False,68,0.74,2020-03-20,0.00,0,5,state,CE,44,0,26.00,26.00,0.00,0.00,13.60,13.60,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78909,NaN,23.00,2021-06-16,202124,"9,187,103.00","9,132,078.00",False,False,856267,"9,320.32",2021-06-16,0.03,21876,458,state,CE,3679,177,"841,261.71","806,123.11","21,432.00","20,623.82","3,392.86","3,665.46",50.00,84.04
79095,NaN,23.00,2021-06-17,202124,"9,187,103.00","9,132,078.00",False,False,858502,"9,344.64",2021-06-17,0.03,21901,459,state,CE,2235,25,"845,180.14","809,820.00","21,507.29","20,710.86","3,918.43","3,696.89",75.29,87.04
79281,NaN,23.00,2021-06-18,202124,"9,187,103.00","9,132,078.00",False,False,861688,"9,379.32",2021-06-18,0.03,21967,460,state,CE,3186,66,"848,502.71","813,380.50","21,581.86","20,790.61","3,322.57","3,560.50",74.57,79.75
79467,NaN,23.00,2021-06-19,202124,"9,187,103.00","9,132,078.00",False,False,864529,"9,410.25",2021-06-19,0.03,22043,461,state,CE,2841,76,"851,997.71","816,928.18","21,665.86","20,869.21","3,495.00","3,547.68",84.00,78.61


In [ ]:
dados_sem_CE = dados_CE.groupby(by = 'Sem_epidemiologica').sum()[['Ultimos_confirmados', 'Ultimas_mortes', 'Novos_casos', 'Novas_mortes']]

In [ ]:
dados_sem_CE['Semana_epi'] = dados_sem_CE.index

In [ ]:
dados_sem_CE

,Ultimos_confirmados,Ultimas_mortes,Novos_casos,Novas_mortes,Semana_epi
Sem_epidemiologica,,,,,
202012,214,0,84,0,202012
202013,1526,10,238,4,202013
202014,3553,92,423,19,202014
202015,9258,365,923,51,202015
202016,16264,882,1394,106,202016
...,...,...,...,...,...
202121,5486009,140746,29859,629,202121
202122,5673519,144810,23889,463,202122
202123,5823663,148689,19438,476,202123


In [ ]:
import datetime
d = "201353"
r = datetime.datetime.strptime(d+'6', '%Y%W%w').date()
print(r)

2014-01-11


In [ ]:
import datetime
d = '2013-W26'
r = datetime.datetime.strptime(d + '-1', '%G-W%V-%u')
print(r)

2013-06-24 00:00:00


In [ ]:
import datetime
week = '201114'
date=datetime.datetime.strptime(week + '-1', "%Y%W-%w")
print(date)

2011-04-04 00:00:00


In [ ]:
dados_sem_CE['Semana_epi'] = pd.to_datetime(dados_sem_CE['Semana_epi'].astype(str)+'5', format= '%Y%W%w')

# Criando as funções

In [ ]:
def prophet_melhor_modelo(dataset, variavel, min_treino = 0.8, min_teste = 0.8, max_teste = 0.95,
                          l_treino = 420, n_periodos =42, v_tempo = 'Data', fer = None, stop = 100):
  if max_teste < min_teste:
    print('ERRO: Máximo de parada não pode ser menor que o mínimo de teste')
    return False

  dados = dataset.copy()
  df = pd.DataFrame()
  df['ds'] = dados[v_tempo][0:l_treino]
  df['y'] = dados[variavel][0:l_treino]

  n = dados.shape[0]
  from fbprophet import Prophet
  import logging
  logging.getLogger('fbprophet').setLevel(logging.WARNING)

  coluna_n_cp = []
  coluna_cp_p_s = []
  coluna_tipo_saz = []
  coluna_cp_r = []
  coluna_r2_treino = []
  coluna_r2_teste = []

  cont = 0
  for cp_p_s in [0.1,0.3,0.5,0.8,1,1.5,2,2.5, 3.0]:
    for n_cp in [5,10,15,20,25,30,35,40,45,50,55,60]:
      for cp_r in [0.8,0.85, 0.9, 0.95]:
        for tipo_saz in ['additive', 'multiplicative']:          
          modelo = Prophet(n_changepoints= n_cp, changepoint_prior_scale= cp_p_s, changepoint_range= cp_r,
                    seasonality_mode= tipo_saz, holidays = fer)
          modelo.fit(df)

          df_futuro = modelo.make_future_dataframe(periods = n_periodos)
          df_previsao = modelo.predict(df_futuro)

          df_teste = pd.DataFrame()
          df_teste['ds'] = dados[v_tempo][l_treino:]
          df_teste['y'] = dados[variavel][l_treino:]

          r2_teste = r2_score(df_teste['y'],df_previsao['yhat'][l_treino:n])
          
          if r2_teste > min_teste:
            r2_treino = r2_score(df['y'],df_previsao['yhat'][0:l_treino])
            if r2_treino > min_treino:
              coluna_n_cp.append(n_cp)
              coluna_cp_p_s.append(cp_p_s)
              coluna_cp_r.append(cp_r)
              coluna_tipo_saz.append(tipo_saz)
              coluna_r2_treino.append(r2_treino)
              coluna_r2_teste.append(r2_teste)
          else:
            cont = cont+1
            if cont == stop:
              print('Atingiu loop máximo sem encontrar os parâmetros requeridos')
              metricas = pd.DataFrame()
              metricas['n_change_points'] = coluna_n_cp
              metricas['change_points_prior_scale'] = coluna_cp_p_s
              metricas['change_point_range'] = coluna_cp_r
              metricas['Sazonality type'] = coluna_tipo_saz
              metricas['r2-score (Treino)'] = coluna_r2_treino
              metricas['r2-score (Teste)'] = coluna_r2_teste
              return metricas.sort_values(by = 'r2-score (Teste)', ascending = False)
          if r2_teste >= max_teste:
            metricas = pd.DataFrame()
            metricas['n_change_points'] = coluna_n_cp
            metricas['change_points_prior_scale'] = coluna_cp_p_s
            metricas['change_point_range'] = coluna_cp_r
            metricas['Sazonality type'] = coluna_tipo_saz
            metricas['r2-score (Treino)'] = coluna_r2_treino
            metricas['r2-score (Teste)'] = coluna_r2_teste
            print('Parâmetros requeridos encontrados')
            return metricas.sort_values(by = 'r2-score (Teste)', ascending = False)

  metricas = pd.DataFrame()
  metricas['n_change_points'] = coluna_n_cp
  metricas['change_points_prior_scale'] = coluna_cp_p_s
  metricas['change_point_range'] = coluna_cp_r
  metricas['Sazonality type'] = coluna_tipo_saz
  metricas['r2-score (Treino)'] = coluna_r2_treino
  metricas['r2-score (Teste)'] = coluna_r2_teste
  print('Parâmetros bons encontrados')
  return metricas.sort_values(by = 'r2-score (Teste)', ascending = False)      

In [ ]:
feriados = [
               '2020-02-25',
               '2020-03-19',
               '2020-03-25',
               '2020-04-10',
               '2020-04-12',
               '2020-04-21',
               '2020-05-01',
               '2020-06-11',
               '2020-08-15',
               '2020-09-07',
               '2020-10-12',
               '2020-11-02',
               '2020-11-15',
               '2020-12-25',
               '2021-01-01',
               '2021-02-15',
               '2021-02-16',
               '2021-02-17',
               '2021-03-19',
               '2021-03-25',
               '2021-04-02',
               '2021-04-04',
               '2021-04-21',
               '2021-05-01'               
]

feriados_CE = pd.DataFrame({
  'holiday': 'Ceara',
  'ds': pd.to_datetime(feriados)})

# Melhores hiperparâmetros

## Estado do Ceará

### Casos novos

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'Novos_casos', min_teste= -0.1, max_teste= 0, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


### Casos acumulados

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'Ultimos_confirmados', min_teste= 0.8, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
10,15,0.10,0.85,additive,1.00,0.95
91,15,0.30,0.85,additive,1.00,0.95
258,15,0.80,0.85,additive,1.00,0.95
594,15,2.50,0.85,additive,1.00,0.95
426,15,1.50,0.85,additive,1.00,0.95
...,...,...,...,...,...,...
359,25,1.00,0.95,multiplicative,1.00,0.80
57,45,0.10,0.95,multiplicative,1.00,0.80
56,45,0.10,0.95,additive,1.00,0.80
107,25,0.30,0.95,multiplicative,1.00,0.80


Melhores hiperparâmetros:

n_change_points = 15

change_points_prior_scale = 0.10

change_point_range = 0.85

Sazonality type = additive


---



### Média móvel dos casos novos (7 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM7_casos_novos', min_teste= -0.1, max_teste= 0, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


Sem bons parâmetros

### Média móvel dos casos novos (28 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM28_casos_novos', min_teste= -0.1, max_teste= 0, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


Sem bons parâmetros

### Média móvel casos acumulados (7 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM7_casos_acumulados', min_teste= 0.8, max_teste= 0.95, fer = feriados_CE)

Parâmetros ótimos encontrados


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
0,5,0.10,0.85,additive,1.00,0.98


n_change_points = 5

change_points_prior_scale = 0.10

change_point_range = 0.85

Sazonality type = additive

r2-score (Treino) = 1.00

r2-score (Teste) = 0.98

### Média móvel casos acumulados (28 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM28_casos_acumulados', min_teste= 0.8, max_teste= 0.95, fer = feriados_CE)

Parâmetros ótimos encontrados


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
2,10,0.10,0.90,additive,1.00,0.99
1,5,0.10,0.90,multiplicative,1.00,0.94
0,5,0.10,0.90,additive,1.00,0.94


n_change_points = 10

change_points_prior_scale = 0.10

change_point_range = 0.90

Sazonality type = additive

r2-score (Treino) = 1.00

r2-score (Teste) = 0.99

### Óbitos acumulados

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'Ultimas_mortes', min_teste= 0.8, max_teste= 0.95, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
32,25,0.30,0.80,additive,1.00,0.94
33,25,0.30,0.80,multiplicative,1.00,0.94
30,20,0.30,0.80,additive,1.00,0.93
31,20,0.30,0.80,multiplicative,1.00,0.93
12,35,0.10,0.80,additive,1.00,0.93
10,30,0.10,0.80,additive,1.00,0.93
22,60,0.10,0.80,additive,1.00,0.93
15,40,0.10,0.80,multiplicative,1.00,0.93
13,35,0.10,0.80,multiplicative,1.00,0.93
9,25,0.10,0.80,multiplicative,1.00,0.93


n_change_points = 25

change_points_prior_scale = 0.3

change_point_range = 0.80

Sazonality type = additive

r2-score (Treino) = 1.00

r2-score (Teste) = 0.94


### Obitos novos

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'Novas_mortes', min_teste= -0.1, max_teste= 0, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


Sem bons parâmetros

### Média móvel óbitos novos (7 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM7_obitos_novos', min_teste= 0.8, max_teste= 0.85, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


Sem bons parâmetros

### Média móvel óbitos novos (28 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM28_obitos_novos', min_teste= 0.8, max_teste= 0.85, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


Sem bons parâmetros

### Média móvel óbitos acumulados (7 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM7_obitos_acumulados', min_teste= 0.8, max_teste= 0.9, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
2,10,0.10,0.85,additive,1.00,0.86
26,10,0.30,0.85,additive,1.00,0.85
3,10,0.10,0.85,multiplicative,1.00,0.85
27,10,0.30,0.85,multiplicative,1.00,0.85
4,15,0.10,0.85,additive,1.00,0.85
6,20,0.10,0.85,additive,1.00,0.85
28,15,0.30,0.85,additive,1.00,0.85
5,15,0.10,0.85,multiplicative,1.00,0.85
7,20,0.10,0.85,multiplicative,1.00,0.85
29,15,0.30,0.85,multiplicative,1.00,0.85


n_change_points = 10

change_points_prior_scale = 0.10

change_point_range = 0.85

Sazonality type = additive

r2-score (Treino) = 1.00

r2-score (Teste) = 0.86

### Média móvel óbitos acumulados (28 dias)

In [ ]:
tabela =  prophet_melhor_modelo(dados_CE,'MM28_obitos_acumulados', min_teste= 0.8, max_teste= 0.95, fer = feriados_CE)

Atingiu loop máximo sem encontrar bons parâmetros


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
53,25,0.30,0.90,multiplicative,1.00,0.93
11,25,0.10,0.90,multiplicative,1.00,0.93
52,25,0.30,0.90,additive,1.00,0.93
10,25,0.10,0.90,additive,1.00,0.93
15,30,0.10,0.90,multiplicative,1.00,0.92
...,...,...,...,...,...,...
8,25,0.10,0.85,additive,1.00,0.82
5,20,0.10,0.85,multiplicative,1.00,0.81
4,20,0.10,0.85,additive,1.00,0.81
43,15,0.30,0.85,multiplicative,1.00,0.81


n_change_points = 25

change_points_prior_scale = 0.3

change_point_range = 0.90

Sazonality type = multipicativo

r2-score (Treino) = 1.00

r2-score (Teste) = 0.93

## Semana epidemiológica

### Casos acumulados

In [ ]:
tabela =  prophet_melhor_modelo(dados_sem_CE,'Ultimos_confirmados', v_tempo= 'Semana_epi', l_treino= 60,
                                min_teste= 0.5, max_teste= 0.6, fer = feriados_CE)

Atingiu loop máximo sem encontrar os parâmetros requeridos


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


### Casos novos

In [ ]:
tabela =  prophet_melhor_modelo(dados_sem_CE,'Novos_casos', v_tempo= 'Semana_epi', l_treino= 60,
                                min_teste= 0.5, max_teste= 0.6, fer = feriados_CE)

Atingiu loop máximo sem encontrar os parâmetros requeridos


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


### Óbitos acumulados

In [ ]:
tabela =  prophet_melhor_modelo(dados_sem_CE,'Ultimas_mortes', v_tempo= 'Semana_epi', l_treino= 60,
                                min_teste= 0.5, max_teste= 0.6, fer = feriados_CE)

Atingiu loop máximo sem encontrar os parâmetros requeridos


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)


### Óbitos novos

In [ ]:
tabela =  prophet_melhor_modelo(dados_sem_CE,'Novas_mortes', v_tempo= 'Semana_epi', l_treino= 60,
                                min_teste= 0.5, max_teste= 0.6, fer = feriados_CE)

Atingiu loop máximo sem encontrar os parâmetros requeridos


In [ ]:
tabela

,n_change_points,change_points_prior_scale,change_point_range,Sazonality type,r2-score (Treino),r2-score (Teste)
